# 第六部分 元编程

## 动态属性和特性

In [2]:
"""
osconfeed.py: Script to download the OSCON schedule feed

# BEGIN OSCONFEED_DEMO

    >>> feed = load()  # <1>
    >>> sorted(feed['Schedule'].keys())  # <2>
    ['conferences', 'events', 'speakers', 'venues']
    >>> for key, value in sorted(feed['Schedule'].items()):
    ...     print('{:3} {}'.format(len(value), key))  # <3>
    ...
      1 conferences
    484 events
    357 speakers
     53 venues
    >>> feed['Schedule']['speakers'][-1]['name']  # <4>
    'Carina C. Zona'
    >>> feed['Schedule']['speakers'][-1]['serial']  # <5>
    141590
    >>> feed['Schedule']['events'][40]['name']
    'There *Will* Be Bugs'
    >>> feed['Schedule']['events'][40]['speakers']  # <6>
    [3471, 5199]


# END OSCONFEED_DEMO
"""

# BEGIN OSCONFEED
from urllib.request import urlopen
import warnings
import os
import json

URL = 'http://www.oreilly.com/pub/sc/osconfeed'
JSON = 'data/osconfeed.json'


def load():
    if not os.path.exists(JSON):
        msg = 'downloading {} to {}'.format(URL, JSON)
        warnings.warn(msg)  # <1>
        with urlopen(URL) as remote, open(JSON, 'wb') as local:  # 在with语句中使用两个上下文管理器（从Python 2.7和Python 3.1起允许这么做），分别用于读取和保存远程文件。
            local.write(remote.read())

    with open(JSON) as fp:
        return json.load(fp)  # <3>

# END OSCONFEED
feed = load()

/var/folders/jh/h9m07pjn1cl8rn6_1czdjjmr0000gn/T/ipykernel_97726/464629970.py:42: UserWarning: downloading http://www.oreilly.com/pub/sc/osconfeed to data/osconfeed.json
  warnings.warn(msg)  # <1>


KeyboardInterrupt: 

### 19.1 使用动态属性访问JSON类数据

feed\['Schedule'\]\['events'\]\[40\]\['name'\]这种句法很冗长。在JavaScript中，可以使用feed.Schedule.events\[40\].name获取那个值。在Python中，可以实现一个近似字典的类（网上有大量实现）。

目标：实现一个FrozenJSON类，只支持读取，即只能访问数据。不过，这个类能递归，自动处理嵌套的映射和列表。

In [20]:
# 目标：实现一个FrozenJSON类，只支持读取，即只能访问数据。不过，这个类能递归，自动处理嵌套的映射和列表。

from collections import abc
import keyword

class FrozenJSON:
    """
        一个只读接口，使用属性表示法访问JSON类对象
    """
    def __init__(self, mapping):
        # self.__data = dict(mapping) # 使用mapping参数构建一个字典。这么做有两个目的：(1)确保传入的是字典（或者是能转换成字典的对象）；(2)安全起见，创建一个副本。
        self.__data = {}
        for key, value in mapping.items():
            if keyword.iskeyword(key): # 检查是否是python的关键字
                key += '_'
            self.__data[key] = value 

    def __getattr__(self, name): # 仅当没有指定名称（name）的属性时才调用__getattr__方法。
        if hasattr(self.__data, name):
            return getattr(self.__data, name) # 如果name是实例属性__data的属性，返回那个属性。调用keys等方法就是通过这种方式处理的。

        else:
            return FrozenJSON.build(self.__data[name]) # 否则，从self.__data中获取name键对应的元素，返回调用FrozenJSON.build（　）方法得到的结果。

    @classmethod
    def build(cls, obj): # 这是一个备选构造方法，@classmethod装饰器经常这么用。
        # Json的数据只有字典和列表两种类型
        if isinstance(obj, abc.Mapping): # 如果obj是映射，那就构建一个FrozenJSON对象。
            return cls(obj)
        elif isinstance(obj, abc.MutableSequence): # 如果是MutableSequence对象，必然是列表，[插图]因此，我们把obj中的每个元素递归地传给.build（　）方法，构建一个列表。
            return [cls.build(item) for item in obj]
        else: # 如果既不是字典也不是列表，那么原封不动地返回元素。
            return obj
    

In [21]:
# 检查 FrozenJSON 类
import json

filepath = 'chapter19/osconfeed.json'

with open(filepath, 'r') as fp:
    j = json.load(fp)

    feed = FrozenJSON(j)

    print(len(feed.Schedule.speakers))

    print(sorted(feed.Schedule.keys()))

    for key, value in sorted(feed.Schedule.items()):
        print('{:3} {}'.format(len(value), key))
    
    fp.close()

357
['conferences', 'events', 'speakers', 'venues']
  1 conferences
484 events
357 speakers
 53 venues


### 19.3 使用__new__方法以灵活的方式创建对象

其实，用于构建实例的是特殊方法__new__：这是个类方法（使用特殊方式处理，因此不必使用@classmethod装饰器），必须返回一个实例。返回的实例会作为第一个参数（即self）传给__init__方法。因为调用__init__方法时要传入实例，而且禁止返回任何值，所以__init__方法其实是“初始化方法”。真正的构造方法是__new__。

我们几乎不需要自己编写__new__方法，因为从object类继承的实现已经足够了。

骚操作的基础：即从__new__方法到__init__方法，是最常见的，但不是唯一的。__new__方法也可以返回其他类的实例，此时，解释器不会调用__init__方法。

In [22]:
# 使用__new__方法取代build方法，构建可能是也可能不是FrozenJSON实例的新对象

from collections import abc
import keyword

class FrozenJSON2:
    """
        一个只读接口，使用属性表示法访问JSON类对象
    """
    def __new__(cls, obj): # __new__是类方法，第一个参数是类本身，余下的参数与__init__方法一样，只不过没有self。
        if isinstance(obj, abc.Mapping): 
            return super().__new__(cls)  #  默认的行为是委托给超类的__new__方法。这里调用的是object基类的__new__方法，把唯一的参数设为FrozenJSON。
        elif isinstance(obj, abc.MutableSequence):  # __new__方法中余下的代码与原先的build方法完全一样。
            return [cls.build(item) for item in obj] 
        else: 
            return obj

    def __init__(self, mapping):
        self.__data = {}
        for key, value in mapping.items():
            if keyword.iskeyword(key): 
                key += '_'
            self.__data[key] = value 

    def __getattr__(self, name):
        if hasattr(self.__data, name):
            return getattr(self.__data, name) 

        else:
            return FrozenJSON(self.__data[name]) # 之前，这里调用的是FrozenJSON.build方法，现在只需调用FrozenJSON构造方法。

    # 使用__new__内置函数代替：
    # @classmethod
    # def build(cls, obj): # 这是一个备选构造方法，@classmethod装饰器经常这么用。
    #     # Json的数据只有字典和列表两种类型
    #     if isinstance(obj, abc.Mapping): # 如果obj是映射，那就构建一个FrozenJSON对象。
    #         return cls(obj)
    #     elif isinstance(obj, abc.MutableSequence): # 如果是MutableSequence对象，必然是列表，[插图]因此，我们把obj中的每个元素递归地传给.build（　）方法，构建一个列表。
    #         return [cls.build(item) for item in obj]
    #     else: # 如果既不是字典也不是列表，那么原封不动地返回元素。
    #         return obj

In [23]:
# 检查 FrozenJSON 类
import json

filepath = 'chapter19/osconfeed.json'

with open(filepath, 'r') as fp:
    j = json.load(fp)

    feed = FrozenJSON2(j)

    print(len(feed.Schedule.speakers))

    print(sorted(feed.Schedule.keys()))

    for key, value in sorted(feed.Schedule.items()):
        print('{:3} {}'.format(len(value), key))

    fp.close()

357
['conferences', 'events', 'speakers', 'venues']
  1 conferences
484 events
357 speakers
 53 venues


### 19.4 使用shelve模块调整OSCON数据源的结构

接下来的任务是，调整数据结构，以便自动获取所链接的记录。

shelve.open高阶函数返回一个shelve.Shelf实例，这是简单的键值对象数据库，背后由dbm模块支持，具有下述特点。
- shelve.Shelf是abc.MutableMapping的子类，因此提供了处理映射类型的重要方法。
- 此外，shelve.Shelf类还提供了几个管理I/O的方法，如sync和close；
- 它也是一个上下文管理器。
- 只要把新值赋予键，就会保存键和值。
- 键必须是字符串。
- 值必须是pickle模块能处理的对象。



In [ ]:
"""
schedule1.py: traversing OSCON schedule data
"""
# BEGIN SCHEDULE1
import warnings
import json

DB_NAME = 'data/schedule1_db'
CONFERENCE = 'conference.115'


def load_osconfeed():
    with open('chapter19/osconfeed.json', 'r') as fp:
        return json.load(fp)


class Record:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)  # <2>


def load_db(db):
    raw_data = load_osconfeed()  # <3>
    warnings.warn('loading ' + DB_NAME)
    for collection, rec_list in raw_data['Schedule'].items():  # <4>
        record_type = collection[:-1]  # <5>
        for record in rec_list:
            key = '{}.{}'.format(record_type, record['serial'])  # <6>
            record['serial'] = key  # <7>
            db[key] = Record(**record)  # <8>

# END SCHEDULE1
